In [ ]:
#Import necessary libraries
import numpy as np
import pandas as pd

#Load the datasets and Create an ExcelFile object
from google.colab import drive
drive.mount('/content/drive')
Data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/RozhinChainStores.xlsx')

Mounted at /content/drive


In [ ]:
#Initial check
print(Data.head(5))
print('\n')
print(Data.dtypes)
print('\n')
print(Data.info())
print('\n')
print(Data.isnull().sum())
print('\n')
print(Data.duplicated().sum())

  شماره فاکتور  تاریخ فروش   کد مشتری                    نام مشتری  \
0            1  1399/02/03  1010005.0  فروشگاه زنجيره اي افق کوروش   
1            1  1399/02/03  1010005.0  فروشگاه زنجيره اي افق کوروش   
2            2  1399/02/03  1010004.0  فروشگاه زنجيره اي افق کوروش   
3            2  1399/02/03  1010004.0  فروشگاه زنجيره اي افق کوروش   
4            3  1399/02/03  1010002.0  فروشگاه زنجيره اي افق کوروش   

               نام فروشگاه  کد فروشنده                       فروشنده  \
0  فروشگاه افق کوروش کرمان   1400059.0  فروشنده فروشگاههاي زنجيره اي   
1  فروشگاه افق کوروش کرمان   1400059.0  فروشنده فروشگاههاي زنجيره اي   
2   فروشگاه افق کوروش مشهد   1400059.0  فروشنده فروشگاههاي زنجيره اي   
3   فروشگاه افق کوروش مشهد   1400059.0  فروشنده فروشگاههاي زنجيره اي   
4  فروشگاه افق کوروش اهواز   1400059.0  فروشنده فروشگاههاي زنجيره اي   

             گروه کالا    کد کالا  \
0               سس گرم  1102001.0   
1  رب گوجه فرنگي روژين  1101002.0   
2  رب گوجه فرنگي روژين  1101002.0  

In [ ]:
#Preprocessing - chose the valuable columns

SelectedColumns = [ 'کد مشتری', 'کد فروشنده', 'کد کالا', 'نام مشتری', 'نام فروشگاه',
                    'فروشنده', 'نام کالا', 'گروه کالا', 'ماه', 'سال', 'مبلغ فروش خالص',
                    'قیمت واحد', 'تعداد فروش و جایزه-ظرف', 'تعداد فروش و جایزه-کارتن' ]

NewData = Data[SelectedColumns]
print(NewData.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80163 entries, 0 to 80162
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   کد مشتری                  80162 non-null  float64
 1   کد فروشنده                80162 non-null  float64
 2   کد کالا                   80162 non-null  float64
 3   نام مشتری                 80162 non-null  object 
 4   نام فروشگاه               80162 non-null  object 
 5   فروشنده                   80162 non-null  object 
 6   نام کالا                  80162 non-null  object 
 7   گروه کالا                 80162 non-null  object 
 8   ماه                       80162 non-null  object 
 9   سال                       80162 non-null  float64
 10  مبلغ فروش خالص            80163 non-null  object 
 11  قیمت واحد                 80163 non-null  object 
 12  تعداد فروش و جایزه-ظرف    80163 non-null  object 
 13  تعداد فروش و جایزه-کارتن  80163 non-null  object 
dtypes: flo

In [ ]:
#Preprocessing - rename the headers

NewData.rename(columns={
               'کد مشتری': 'CustomerCode',
               'کد فروشنده': 'SellerCode',
               'کد کالا': 'ProductCode',
               'نام مشتری': 'CustomerName',
               'نام فروشگاه': 'StoreName',
               'فروشنده': 'SellerName',
               'نام کالا': 'ProductName',
               'گروه کالا': 'ProductGroup',
               'ماه': 'Month',
               'سال': 'Year',
               'مبلغ فروش خالص': 'NetSales',
               'قیمت واحد': 'UnitPrice',
               'تعداد فروش و جایزه-ظرف': 'Units',
               'تعداد فروش و جایزه-کارتن': 'Cartons'
               }, inplace=True)

print(NewData.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80163 entries, 0 to 80162
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   CustomerCode  80162 non-null  float64
 1   SellerCode    80162 non-null  float64
 2   ProductCode   80162 non-null  float64
 3   CustomerName  80162 non-null  object 
 4   StoreName     80162 non-null  object 
 5   SellerName    80162 non-null  object 
 6   ProductName   80162 non-null  object 
 7   ProductGroup  80162 non-null  object 
 8   Month         80162 non-null  object 
 9   Year          80162 non-null  float64
 10  NetSales      80163 non-null  object 
 11  UnitPrice     80163 non-null  object 
 12  Units         80163 non-null  object 
 13  Cartons       80163 non-null  object 
dtypes: float64(4), object(10)
memory usage: 8.6+ MB
None


<ipython-input-4-69ceff86f8b6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NewData.rename(columns={


In [ ]:
#Preprocessing - function to normalize Persian text

import re

#Function to manage the text
def normalize_persian(text):
    if not isinstance(text, str):
        return None                     #Handle non-string values
    text = text.strip()                 #Remove leading/trailing spaces
    text = re.sub(r'\s+', ' ', text)    #Replace multiple spaces with one
    text = text.replace('ي', 'ی')       #Convert Arabic "ي" to Persian "ی"
    text = text.replace('ك', 'ک')       #Convert Arabic "ك" to Persian "ک"
    return text

PersianMonths = {'فروردین': '01','اردیبهشت': '02','خرداد': '03','تیر': '04',
                 'مرداد': '05','شهریور': '06','مهر': '07','آبان': '08',
                 'آذر': '09','دی': '10','بهمن': '11','اسفند': '12'}

#Normalize and debug 'Month' column
NewData['Month'] = NewData['Month'].apply(normalize_persian)
NewData['MonthNumeric'] = NewData['Month'].map(PersianMonths)
NewData['MonthNumeric'].fillna('Unknown', inplace=True)
NewData = NewData[NewData['MonthNumeric'] != 'Unknown']

In [ ]:
#Check data
print("Unique values in 'Month':")
print(NewData['MonthNumeric'].unique())
print(NewData.info())

Unique values in 'Month':
['02' '03' '04' '05' '06' '07' '08' '09' '10' '11' '12' '01']
<class 'pandas.core.frame.DataFrame'>
Index: 80162 entries, 0 to 80161
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   CustomerCode  80162 non-null  float64
 1   SellerCode    80162 non-null  float64
 2   ProductCode   80162 non-null  float64
 3   CustomerName  80162 non-null  object 
 4   StoreName     80162 non-null  object 
 5   SellerName    80162 non-null  object 
 6   ProductName   80162 non-null  object 
 7   ProductGroup  80162 non-null  object 
 8   Month         80162 non-null  object 
 9   Year          80162 non-null  float64
 10  NetSales      80162 non-null  object 
 11  UnitPrice     80162 non-null  object 
 12  Units         80162 non-null  object 
 13  Cartons       80162 non-null  object 
 14  MonthNumeric  80162 non-null  object 
dtypes: float64(4), object(11)
memory usage: 9.8+ MB
None


In [ ]:
#Preprocessing - reorder rows by year

NewData.sort_values(by='Year', ascending=True, inplace=True)
print(NewData['Year'].head(30))

0       1399.0
8220    1399.0
8221    1399.0
8222    1399.0
8223    1399.0
8224    1399.0
8225    1399.0
8226    1399.0
8227    1399.0
8228    1399.0
8219    1399.0
8229    1399.0
8231    1399.0
8232    1399.0
8233    1399.0
8234    1399.0
8235    1399.0
8236    1399.0
8237    1399.0
8238    1399.0
8239    1399.0
8230    1399.0
8240    1399.0
8218    1399.0
8216    1399.0
8196    1399.0
8197    1399.0
8198    1399.0
8199    1399.0
8200    1399.0
Name: Year, dtype: float64


In [ ]:
#Preprocessing - define numeric codes for unique values

NewData['StoreCode'] = NewData['StoreName'].astype('category').cat.codes
NewData['ProductGroupCode'] = NewData['ProductGroup'].astype('category').cat.codes
print(NewData.info())

<class 'pandas.core.frame.DataFrame'>
Index: 80162 entries, 0 to 80161
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CustomerCode      80162 non-null  float64
 1   SellerCode        80162 non-null  float64
 2   ProductCode       80162 non-null  float64
 3   CustomerName      80162 non-null  object 
 4   StoreName         80162 non-null  object 
 5   SellerName        80162 non-null  object 
 6   ProductName       80162 non-null  object 
 7   ProductGroup      80162 non-null  object 
 8   Month             80162 non-null  object 
 9   Year              80162 non-null  float64
 10  NetSales          80162 non-null  object 
 11  UnitPrice         80162 non-null  object 
 12  Units             80162 non-null  object 
 13  Cartons           80162 non-null  object 
 14  MonthNumeric      80162 non-null  object 
 15  StoreCode         80162 non-null  int16  
 16  ProductGroupCode  80162 non-null  int8   
dty

In [ ]:
#Preprocessing - remove dots in specific columns

NewData['Cartons'] = NewData['Cartons'].astype(str).str.replace('.', '', regex=False)
NewData['Units'] = NewData['Units'].astype(str).str.replace('.', '', regex=False)

print(NewData[['Cartons', 'Units']].head(10))

     Cartons Units
0         48   288
8220       4    24
8221      10   120
8222       4    48
8223       5    60
8224      40   480
8225       5    60
8226       4    48
8227      60   720
8228      60   720


In [ ]:
#Preprocessing - change types of numeric columns

NumericColumns = ['NetSales', 'UnitPrice', 'Units', 'Cartons']

for column in NumericColumns:
    NewData[column] = pd.to_numeric(NewData[column], errors='coerce')

print(NewData.info())

<class 'pandas.core.frame.DataFrame'>
Index: 80162 entries, 0 to 80161
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CustomerCode      80162 non-null  float64
 1   SellerCode        80162 non-null  float64
 2   ProductCode       80162 non-null  float64
 3   CustomerName      80162 non-null  object 
 4   StoreName         80162 non-null  object 
 5   SellerName        80162 non-null  object 
 6   ProductName       80162 non-null  object 
 7   ProductGroup      80162 non-null  object 
 8   Month             80162 non-null  object 
 9   Year              80162 non-null  float64
 10  NetSales          80162 non-null  int64  
 11  UnitPrice         80162 non-null  int64  
 12  Units             80162 non-null  int64  
 13  Cartons           80162 non-null  int64  
 14  MonthNumeric      80162 non-null  object 
 15  StoreCode         80162 non-null  int16  
 16  ProductGroupCode  80162 non-null  int8   
dty

In [ ]:
#Preprocessing - keep identifiers as strings

IdentifierColumns = ['CustomerCode', 'SellerCode', 'ProductCode', 'StoreCode', 'ProductGroupCode']

for column in IdentifierColumns:
    NewData[column] = NewData[column].astype(str)

print(NewData.info())

<class 'pandas.core.frame.DataFrame'>
Index: 80162 entries, 0 to 80161
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CustomerCode      80162 non-null  object 
 1   SellerCode        80162 non-null  object 
 2   ProductCode       80162 non-null  object 
 3   CustomerName      80162 non-null  object 
 4   StoreName         80162 non-null  object 
 5   SellerName        80162 non-null  object 
 6   ProductName       80162 non-null  object 
 7   ProductGroup      80162 non-null  object 
 8   Month             80162 non-null  object 
 9   Year              80162 non-null  float64
 10  NetSales          80162 non-null  int64  
 11  UnitPrice         80162 non-null  int64  
 12  Units             80162 non-null  int64  
 13  Cartons           80162 non-null  int64  
 14  MonthNumeric      80162 non-null  object 
 15  StoreCode         80162 non-null  object 
 16  ProductGroupCode  80162 non-null  object 
dty

In [ ]:
#Preprocessing - convert year and month columns to integers

NewData['Year'] = NewData['Year'].astype(int)
NewData['MonthNumeric'] = NewData['MonthNumeric'].astype(int)

print(NewData.info())

<class 'pandas.core.frame.DataFrame'>
Index: 80162 entries, 0 to 80161
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   CustomerCode      80162 non-null  object
 1   SellerCode        80162 non-null  object
 2   ProductCode       80162 non-null  object
 3   CustomerName      80162 non-null  object
 4   StoreName         80162 non-null  object
 5   SellerName        80162 non-null  object
 6   ProductName       80162 non-null  object
 7   ProductGroup      80162 non-null  object
 8   Month             80162 non-null  object
 9   Year              80162 non-null  int64 
 10  NetSales          80162 non-null  int64 
 11  UnitPrice         80162 non-null  int64 
 12  Units             80162 non-null  int64 
 13  Cartons           80162 non-null  int64 
 14  MonthNumeric      80162 non-null  int64 
 15  StoreCode         80162 non-null  object
 16  ProductGroupCode  80162 non-null  object
dtypes: int64(6), obje

In [ ]:
#Check data

print(NewData.isnull().sum())
print('\n')
print(NewData.duplicated().sum())

DuplicatesAllColumns = NewData[NewData.duplicated()]

KeyColumns = ['CustomerCode', 'SellerCode', 'ProductCode', 'Month', 'Year']
DuplicatesKeyColumns = NewData[NewData.duplicated(subset=KeyColumns)]

print("Duplicates based on all columns:")
print(DuplicatesAllColumns)

print("\nDuplicates based on selected key columns:")
print(DuplicatesKeyColumns)

CustomerCode        0
SellerCode          0
ProductCode         0
CustomerName        0
StoreName           0
SellerName          0
ProductName         0
ProductGroup        0
Month               0
Year                0
NetSales            0
UnitPrice           0
Units               0
Cartons             0
MonthNumeric        0
StoreCode           0
ProductGroupCode    0
dtype: int64


4690
Duplicates based on all columns:
      CustomerCode SellerCode ProductCode                   CustomerName  \
8113     1010113.0  1400092.0   1101002.0                      حسن شگفتي   
8114     1010113.0  1400092.0   1101006.0                      حسن شگفتي   
8116     1010113.0  1400092.0   1102001.0                      حسن شگفتي   
8316     1010101.0  1400072.0   1102005.0  فروشگاه زنجيره اي فاميلي مدرن   
8317     1010101.0  1400072.0   1101002.0  فروشگاه زنجيره اي فاميلي مدرن   
...            ...        ...         ...                            ...   
69002    1010615.0  1400063.0   1101008.0

In [ ]:
#Save the data to a dataframe and new csv file

FilePath = '/content/drive/MyDrive/Colab Notebooks/Chain-5Years-Cleaned.csv'
NewData.to_csv(FilePath, index=False)